<a href="https://colab.research.google.com/github/Ataerman/LLM_RAG-AI_AGENT/blob/main/Rag_basic_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langchain-openai langchain-community langchain langchain-tavily
!pip install  langchain-cohere \
sentence-transformers faiss-cpu \
datasets accelerate \
cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6

In [2]:
from google.colab import userdata
import os
openai_api_key=userdata.get('openai')
cohere_api_key=userdata.get('cohere')
langsmith_api_key=userdata.get('langsmith')
tavily_api_key=userdata.get('tavily')

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = langsmith_api_key
os.environ["LANGSMITH_PROJECT"] = "langgraph_first"


In [151]:

from typing import TypedDict, List, Literal
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.types import Command

text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics.
Since its premiere, Interstellar gained a cult following, and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""

doc = Document(page_content=text)
splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = splitter.split_documents([doc])
docs = [Document(**d) if isinstance(d, dict) else d for d in docs]

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 2. Durum yapısı
class RagState(TypedDict):
    messages: List[HumanMessage | AIMessage]
    docs: List[Document]

# 3. Retriever Agent
def retriever_agent(state: RagState) -> Command:
    query = state["messages"][-1].content
    docs = retriever.invoke(query)
    docs = [Document(**d) if isinstance(d, dict) else d for d in docs]
    return Command(goto="reranker_agent", update={"docs": docs})

# 4. Reranker Agent (Cohere)
cohere_api_key = userdata.get("cohere")
cohere_reranker = ContextualCompressionRetriever(
    base_compressor=CohereRerank(
        cohere_api_key=cohere_api_key,
        model="rerank-multilingual-v3.0",
        top_n=3
    ),
    base_retriever=retriever
)


def reranker_agent(state: RagState) -> Command:
    query = state["messages"][-1].content
    fixed_docs = [Document(**d) if isinstance(d, dict) else d for d in state["docs"]]
    reranked_docs = cohere_reranker.get_relevant_documents(query)
    return Command(
        goto="answer_agent",
        update={"docs": reranked_docs})

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3, openai_api_key=openai_api_key)

def answer_agent(state: RagState) -> Command:
    query = state["messages"][-1].content
    context = "\n\n".join([doc.page_content for doc in state["docs"]])
    prompt = f"Context:\n{context}\n\nQuestion: {query}"
    response = llm.invoke(prompt)
    return Command(goto=END, update={"messages": [AIMessage(content=response.content)]})


rag_graph = (
    StateGraph(RagState)
    .add_node("retriever_agent", retriever_agent)
    .add_node("reranker_agent", reranker_agent)
    .add_node("answer_agent", answer_agent)
    .add_edge("retriever_agent", "reranker_agent")
    .add_edge("reranker_agent", "answer_agent")
    .add_edge("answer_agent", END)
    .set_entry_point("retriever_agent")
    .compile()
)


In [152]:
from IPython.display import Image,display
print(rag_graph.get_graph().draw_mermaid())



---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retriever_agent(retriever_agent)
	reranker_agent(reranker_agent)
	answer_agent(answer_agent)
	__end__([<p>__end__</p>]):::last
	__start__ --> retriever_agent;
	reranker_agent --> answer_agent;
	retriever_agent --> reranker_agent;
	answer_agent --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [153]:

# 7. Çalıştırma
if __name__ == "__main__":
    user_input = input("\u2753 Soru: ")
    state = {"messages": [HumanMessage(content=user_input)], "docs": []}
    result = rag_graph.invoke(state)

    print("\n\U0001f4e4 Yanıt:")
    for m in result["messages"]:
        print(f"{m.type.upper()}: {m.content}")


❓ Soru: Who is directed Interstellar?

📤 Yanıt:
AI: Interstellar was directed by Christopher Nolan.
